In [44]:
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
import re

In [37]:
inflow = pd.read_parquet('/uss/hdsi-prismdata/q1-ucsd-inflows.pqt')
outflow = pd.read_parquet('/uss/hdsi-prismdata/q1-ucsd-outflows.pqt')

In [95]:
memos = pd.Series(outflow[outflow['category'] != outflow['memo']]['memo'].unique())
memos

0                 TST* Casa Del Rio - Exp Fairlawn OH 09/24
1                                        Buffalo Wild Wings
2                                           Oculus CA 04/16
3                               LOS GIRASOLES STOW OH 03/08
4                                 BUZZIS LAUNDRY 1 OH 03/28
                                ...                        
528761    DEBIT CARD WITHDRAWAL PURCHASEAudible*Z295DXXX...
528762    DEBIT CARD WITHDRAWAL PURCHASEJACK IN THE BOX ...
528763    DEBIT CARD WITHDRAWAL PURCHASEAmazon Prime*TI4...
528764    POS WITHDRAWALAZ LOT QUIKTRIP XXXX XXXX E INDI...
528765    POS WITHDRAWALWAL-MART #XXXX XXXX E MCKELLIPS ...
Length: 528766, dtype: object

In [78]:
# for memo in memos:
#     print(memo)

In [79]:
# pattern = r"X{2,}"
# matches = [s for s in memos if re.search(pattern, s)]
# matches

In [ ]:
memos_clean = memos.str.lower()
memos_clean = memos_clean.apply(lambda x: re.sub(r'[^a-z0-9\s]', '', x)) # Remove special characters
memos_clean = memos_clean.apply(lambda x: re.sub(r'\s+', ' ', x).strip()) # Remove whitespace
print(memos)
print(memos_clean)

In [ ]:
vectorizer = CountVectorizer(ngram_range=(5, 5), min_df=10)
X = vectorizer.fit_transform(memos_clean)

term_counts = X.sum(axis=0)
frequencies = [(term, term_counts[0, idx]) for term, idx in vectorizer.vocabulary_.items()]

sorted_frequencies = sorted(frequencies, key=lambda x: x[1], reverse=True)

In [ ]:
for term, freq in sorted_frequencies[:10]:
    print(f'"{term}": {freq}')

In [152]:
memos_clean[memos_clean.str.contains('xxxxxxxxxx ca sxxxxxxxxxxxxxxx card xxxx')]

4190      purchase authorized on 08/30 cash app*benjamin...
4789      purchase authorized on 12/20 cash app*nakia w ...
4793      purchase authorized on 03/29 cash app*kaylen x...
4794      purchase authorized on 04/19 cash app*ktr xxxx...
4800      purchase authorized on 02/26 cash app*kaylen x...
                                ...                        
525149    purchase authorized on 10/24 cash app*elizabet...
525207    purchase authorized on 11/08 cash app*elizabet...
525275    purchase authorized on 12/01 cash app*elizabet...
525342    purchase authorized on 12/20 cash app*abby eri...
525387    purchase authorized on 01/04 cash app*elizabet...
Length: 5885, dtype: object

In [149]:
memos_clean[memos_clean.str.lower().str.contains('visa check card')]

7828      pos debit - visa check card xxxx - amazon.com*...
7830      pos debit - visa check card xxxx - instacart*1...
7834      pos debit - visa check card xxxx - amazon.com*...
7839      pos debit - visa check card xxxx - amazon.com*...
7841      pos debit - visa check card xxxx - amazon.com*...
                                ...                        
526419    pos debit - visa check card xxxx - mcdonald's ...
526420    pos debit - visa check card xxxx - fast-fix je...
526421    pos debit - visa check card xxxx - lowe's #xxx...
526422    pos debit - visa check card xxxx - target t- x...
526423    pos debit - visa check card xxxx - sam's club ...
Length: 11667, dtype: object

In [62]:
# for memo in pd.Series(memos).str.split():
#     if '/' in memo[0]:
#         print(memo)